Example of doing Vector Search in Neo4j
In prepration, create an account in the Neo4j Aura cloud service and obtain the required credentials for connecting.
Below we would create a DB. Free tier provides you with one DB.

In [ ]:
!pip install neo4j openai tiktoken datasets

In [ ]:
# Prepare to prompt for input
from getpass import getpass
import os

In [ ]:
# Enter hostname for the Neo4j Aura database
url = getpass('Enter Neo4j Aura DB hostname: ')

In [ ]:
# Enter username for the Neo4j Aura database
username = getpass('Enter Neo4j Aura DB username: ')

In [ ]:
# Enter password for the Neo4j Aura database
password = getpass('Enter Neo4j Aura DB password: ')

In [ ]:
# Enter the OpenAI api key
os.environ["OPENAI_API_KEY"] = getpass('Enter your OpenAI API key: ')

In [ ]:
# Use Langchain since it's easier to use for Neo4j vector search
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Neo4jVector
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

In [ ]:
# Load some data
# Get from https://github.com/langchain-ai/langchain/blob/master/docs/extras/modules/state_of_the_union.txt

loader = TextLoader("./state_of_the_union.txt")
dataset = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(dataset)

embeddings = OpenAIEmbeddings()

In [ ]:
# Connect to Neo4j Aura with the Langchain Neo4jVector module and create a vector index if needed.

db = Neo4jVector.from_documents(
    documents=docs, embedding=embeddings, url=url, username=username, password=password,
)

In [ ]:
# Query the Neo4j Aura database for similar documents
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query, k=2)

In [ ]:
# Loop through the results and print them
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)